## **Setting up Development Environment**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/MyDrive/Colab Notebooks/Hate-Speech-Detection-in-Tamil/HASOC Tamil/HL

/content/drive/MyDrive/Colab Notebooks/Hate-Speech-Detection-in-Tamil/HASOC Tamil/HL


In [3]:
! pip install -U -q transformers
! pip install -U -q sentencepiece

## **Importing Libraries**

In [24]:
import pandas as pd

import numpy as np

import re

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from transformers import AutoTokenizer, AutoModel

import pickle

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## **Loading Data**

In [5]:
dataframe = pd.read_csv("Dataset/Tamil-Codemixed_offensive_data_Training-Tweet-HL.csv")

In [6]:
dataframe.head()

,id,text,category
0,TA_HL100,Iyaooo Kovam pattutene sothula visatha vachuru...,NOT
1,TA_HL101,@Asha Apo neenga atha government ku theriya pa...,NOT
2,TA_HL102,@Bala sundar ayyo sorry...antha line ah explai...,NOT
3,TA_HL105,@kalimuthu ne ena lusa...yaaru edhu panaalum e...,NOT
4,TA_HL109,1st baby ku neat ah feed panunga plzz ipdi iru...,NOT


In [7]:
dataframe.describe()

,id,text,category
count,4000,4000,4000
unique,3999,3657,2
top,TA_TW1600,Adei ellarum ena pei ah da antha time la veliy...,NOT
freq,2,2,2020


In [8]:
dataframe['category'].unique()

array(['NOT', 'OFF'], dtype=object)

In [9]:
dataframe.drop_duplicates(subset ="id",keep = False, inplace = True)
dataframe.describe()

,id,text,category
count,3998,3998,3998
unique,3998,3656,2
top,TA_TW1750,Adei ellarum ena pei ah da antha time la veliy...,NOT
freq,1,2,2020


In [10]:
dataframe.shape

(3998, 3)

In [11]:
text = dataframe['text']
text

0       Iyaooo Kovam pattutene sothula visatha vachuru...
1       @Asha Apo neenga atha government ku theriya pa...
2       @Bala sundar ayyo sorry...antha line ah explai...
3       @kalimuthu ne ena lusa...yaaru edhu panaalum e...
4       1st baby ku neat ah feed panunga plzz ipdi iru...
                              ...                        
3995    Yaroda body structure semaya irukum? Sema mood...
3996    Yenda naangala politics varom nu pala varusham...
3997    Yepdithan seruppala adichalum arasiyalvathikku...
3998    @USER Paithiyam ena unga vanthu full ah forwar...
3999    RT @USER : Itha vidaa kevalam veraa irukaa vij...
Name: text, Length: 3998, dtype: object

In [12]:
label = dataframe['category']
label

0       NOT
1       NOT
2       NOT
3       NOT
4       NOT
       ... 
3995    OFF
3996    OFF
3997    OFF
3998    OFF
3999    OFF
Name: category, Length: 3998, dtype: object

## **Data Preprocessing**

In [13]:
le = LabelEncoder()
label = le.fit_transform(label)

In [14]:
english_stopwords = stopwords.words("english")
english_stemmer = SnowballStemmer("english")

def clean_text(text):
    text = re.sub(r"@[A-Za-z0-9]+",' ', text)
    text = re.sub(r"#[A-Za-z0-9]+",' ', text)
    text = re.sub(r"https?://[A-Za-z0-9./]+",' ', text)
    text = re.sub("RT ", " ", text)
    text = re.sub("\n", " ", text)
    text = re.sub(r" +", " ", text)
    tokens = []
    for token in text.split():
        if token not in english_stopwords:
            token = english_stemmer.stem(token)
            tokens.append(token)
    return " ".join(tokens)

text = [clean_text(i) for i in text]

text = np.array(text)

## **Feature Extraction**

In [15]:
# Load Transformer Model

tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
model = AutoModel.from_pretrained("ai4bharat/indic-bert")

Downloading:   0%|          | 0.00/507 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.38M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/129M [00:00<?, ?B/s]

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertModel: ['sop_classifier.classifier.weight', 'predictions.dense.bias', 'sop_classifier.classifier.bias', 'predictions.bias', 'predictions.decoder.weight', 'predictions.LayerNorm.bias', 'predictions.LayerNorm.weight', 'predictions.dense.weight', 'predictions.decoder.bias']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:
# Sample Output

tokenized_input = tokenizer(
        text[0],
        padding=True,
        truncation=False,
        return_tensors='pt'
    )

sample_output = model(**tokenized_input)

In [19]:
print(tokenized_input)

{'input_ids': tensor([[     2,      8,  13690,   1435,  26926,  39649, 150789,   2804, 163243,
          22814,    870,    778,  41422,  40725,  29756,  10048,  27352,  48355,
          34003, 166753,      5,      9,  26436,  26926,    113,      3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1]])}


In [20]:
print(sample_output)

BaseModelOutputWithPooling(last_hidden_state=tensor([[[-0.0134, -0.0114, -0.0193,  ..., -0.0295, -0.0122, -0.0224],
         [-0.9311,  0.3630,  0.1045,  ..., -0.1867, -0.2342,  0.1677],
         [ 0.5631, -0.2697, -0.1451,  ...,  0.0476,  0.1300, -0.1667],
         ...,
         [ 0.1645, -0.1783, -0.2019,  ..., -0.0315,  0.1328, -0.0804],
         [ 0.2489, -0.2253, -0.0331,  ..., -0.0477, -0.0136, -0.4301],
         [-0.0134, -0.0114, -0.0193,  ..., -0.0295, -0.0122, -0.0224]]],
       grad_fn=<NativeLayerNormBackward>), pooler_output=tensor([[-5.1553e-02,  5.1777e-02,  1.5507e-02, -2.9941e-03,  4.9526e-02,
          8.2935e-02,  3.9304e-02, -1.1371e-02, -1.0891e-02,  6.0344e-02,
         -7.6168e-03,  4.5780e-02,  1.4764e-02, -1.7554e-02,  5.2142e-02,
         -1.6777e-02, -4.4734e-02,  7.2887e-04, -1.2094e-01, -8.1995e-02,
          7.9113e-02,  7.5058e-02, -3.2143e-02, -9.8400e-02,  8.8309e-04,
         -3.5356e-02, -5.5147e-02, -2.1417e-03,  2.2768e-02,  7.8443e-03,
          2.

In [21]:
print(sample_output.pooler_output.cpu().detach().numpy().shape)

(1, 768)


In [22]:
# Tokenizing Input Data

input = []

for i in text:
    tokenized_input = tokenizer(
        i,
        padding=True,
        truncation=False,
        return_tensors='pt'
    )
    input.append(tokenized_input)

In [25]:
# Embedding the Input Data

output = []

j = 1

for i in range(len(input)):
    
    model_output = model(**input[i])
    model_output = model_output.pooler_output.cpu().detach().numpy()
    output.append(model_output)
    
    # Saving Embedded Input Data to Disk

    if ((i + 1) % 200) == 0:
        
        file_name = "./IndicBert-Embedded-TrainData/output" + str(j) + ".pickle"
        
        with open(file_name, "wb") as fp:   
            # Pickling
            pickle.dump(output, fp)
        
        print(file_name + " done")
        output = []
        j += 1
        
file_name = "./IndicBert-Embedded-TrainData/output" + str(j) + ".pickle"

with open(file_name, "wb") as fp:   
    # Pickling
    pickle.dump(output, fp)

print(file_name + " done")
output = []

./IndicBert-Embedded-TrainData/output1.pickle done
./IndicBert-Embedded-TrainData/output2.pickle done
./IndicBert-Embedded-TrainData/output3.pickle done
./IndicBert-Embedded-TrainData/output4.pickle done
./IndicBert-Embedded-TrainData/output5.pickle done
./IndicBert-Embedded-TrainData/output6.pickle done
./IndicBert-Embedded-TrainData/output7.pickle done
./IndicBert-Embedded-TrainData/output8.pickle done
./IndicBert-Embedded-TrainData/output9.pickle done
./IndicBert-Embedded-TrainData/output10.pickle done
./IndicBert-Embedded-TrainData/output11.pickle done
./IndicBert-Embedded-TrainData/output12.pickle done
./IndicBert-Embedded-TrainData/output13.pickle done
./IndicBert-Embedded-TrainData/output14.pickle done
./IndicBert-Embedded-TrainData/output15.pickle done
./IndicBert-Embedded-TrainData/output16.pickle done
./IndicBert-Embedded-TrainData/output17.pickle done
./IndicBert-Embedded-TrainData/output18.pickle done
./IndicBert-Embedded-TrainData/output19.pickle done
./IndicBert-Embedded-

In [26]:
# Loading Embedded Input Data from Disk

output = []

for i in range(20):
    file_name = "./IndicBert-Embedded-TrainData/output" + str(i + 1) + ".pickle"
    with open(file_name, "rb") as fp:   #Pickling
        file_output = pickle.load(fp)
        for x in file_output:
            output.append(x)
    print(file_name + " done")

X = output
output = []

./IndicBert-Embedded-TrainData/output1.pickle done
./IndicBert-Embedded-TrainData/output2.pickle done
./IndicBert-Embedded-TrainData/output3.pickle done
./IndicBert-Embedded-TrainData/output4.pickle done
./IndicBert-Embedded-TrainData/output5.pickle done
./IndicBert-Embedded-TrainData/output6.pickle done
./IndicBert-Embedded-TrainData/output7.pickle done
./IndicBert-Embedded-TrainData/output8.pickle done
./IndicBert-Embedded-TrainData/output9.pickle done
./IndicBert-Embedded-TrainData/output10.pickle done
./IndicBert-Embedded-TrainData/output11.pickle done
./IndicBert-Embedded-TrainData/output12.pickle done
./IndicBert-Embedded-TrainData/output13.pickle done
./IndicBert-Embedded-TrainData/output14.pickle done
./IndicBert-Embedded-TrainData/output15.pickle done
./IndicBert-Embedded-TrainData/output16.pickle done
./IndicBert-Embedded-TrainData/output17.pickle done
./IndicBert-Embedded-TrainData/output18.pickle done
./IndicBert-Embedded-TrainData/output19.pickle done
./IndicBert-Embedded-

In [27]:
X = np.array(X)
X.shape

(3998, 1, 768)

In [28]:
X = X.reshape(3998, 768)
X.shape

(3998, 768)

In [29]:
y = label
y.shape

(3998,)

## **Train Test Split**

In [30]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [31]:
X_train.shape

(3198, 768)

In [32]:
X_val.shape

(800, 768)

## **Logistic Regression**

In [33]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_val)
print(classification_report(y_val, lr_pred))

              precision    recall  f1-score   support

           0       0.61      0.67      0.64       420
           1       0.59      0.52      0.56       380

    accuracy                           0.60       800
   macro avg       0.60      0.60      0.60       800
weighted avg       0.60      0.60      0.60       800



## **Support Vector Machine**

In [34]:
svm = SVC()
svm.fit(X_train, y_train)
svm_pred = svm.predict(X_val)
print(classification_report(y_val, svm_pred))

              precision    recall  f1-score   support

           0       0.59      0.79      0.67       420
           1       0.62      0.39      0.48       380

    accuracy                           0.60       800
   macro avg       0.61      0.59      0.58       800
weighted avg       0.61      0.60      0.58       800



## **Naive Bayes**

In [35]:
nb = GaussianNB()
nb.fit(X_train, y_train)
nb_pred = nb.predict(X_val)
print(classification_report(y_val, nb_pred))

              precision    recall  f1-score   support

           0       0.60      0.72      0.66       420
           1       0.60      0.47      0.53       380

    accuracy                           0.60       800
   macro avg       0.60      0.59      0.59       800
weighted avg       0.60      0.60      0.59       800



## **Stochastic Gradient Descent**

In [36]:
sgd = SGDClassifier()
sgd.fit(X_train, y_train)
sgd_pred = sgd.predict(X_val)
print(classification_report(y_val, sgd_pred))

              precision    recall  f1-score   support

           0       0.59      0.84      0.69       420
           1       0.67      0.36      0.46       380

    accuracy                           0.61       800
   macro avg       0.63      0.60      0.58       800
weighted avg       0.63      0.61      0.58       800



## **K Nearest Neighbours**

In [37]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_val)
print(classification_report(y_val, knn_pred))

              precision    recall  f1-score   support

           0       0.62      0.57      0.59       420
           1       0.56      0.61      0.59       380

    accuracy                           0.59       800
   macro avg       0.59      0.59      0.59       800
weighted avg       0.59      0.59      0.59       800



## **Decision Tree**

In [38]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
dt_pred = dt.predict(X_val)
print(classification_report(y_val, dt_pred))

              precision    recall  f1-score   support

           0       0.65      0.64      0.64       420
           1       0.61      0.63      0.62       380

    accuracy                           0.63       800
   macro avg       0.63      0.63      0.63       800
weighted avg       0.63      0.63      0.63       800



## **Random Forest**

In [39]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_val)
print(classification_report(y_val, rf_pred))

              precision    recall  f1-score   support

           0       0.75      0.71      0.73       420
           1       0.70      0.74      0.72       380

    accuracy                           0.72       800
   macro avg       0.72      0.72      0.72       800
weighted avg       0.73      0.72      0.72       800



## **Majority Voting**

In [40]:
mv_pred = []

for i in range(len(lr_pred)):
    one = 0
    zero = 0
    predictions = [lr_pred[i], svm_pred[i], rf_pred[i], sgd_pred[i], 
                   nb_pred[i], knn_pred[i], dt_pred[i]]
    for pred in predictions:
        if pred == 1: one += 1
        if pred == 0: zero +=1
    if one > zero: mv_pred.append(1)
    else: mv_pred.append(0)

mv_pred = np.array(mv_pred)

print(classification_report(y_val, mv_pred))

              precision    recall  f1-score   support

           0       0.61      0.76      0.68       420
           1       0.64      0.47      0.54       380

    accuracy                           0.62       800
   macro avg       0.63      0.62      0.61       800
weighted avg       0.63      0.62      0.61       800

